In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict

In [2]:
llm = ChatOllama(model="llama3.2")

In [17]:
class BlogState(TypedDict):
    outline: str
    blog: str
    topic: str
    rating: float

In [4]:
def create_outline(state: BlogState)-> BlogState:
    topic= state["topic"]
    outline = llm.invoke(f"Create an outline for a blog post on {topic}").content
    state["outline"] = outline
    return state
    

In [7]:
def write_blog(state: BlogState)->BlogState:
    outline = state["outline"]
    topic = state["topic"]
    prompt=f"Write a blog post on the title - {topic} using the following outline {outline}"
    blog=llm.invoke(prompt).content
    state["blog"] = blog
    return state


In [ ]:
def blog_rating(state:BlogState)->BlogState:
    outline=state["outline"]
    blog= state["blog"]
    prompt=f"Base of this {outline} rate my blog {blog} and generate a integer score from 10. Only provide an integer number"

    rating= llm.invoke(prompt).content

    state["rating"]=rating
    return state

In [31]:
# create graph
graph = StateGraph(BlogState)
# create the nodes
graph.add_node("create_outline", create_outline)
graph.add_node("write_blog", write_blog)
graph.add_node("rate_blog", blog_rating)

# create edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "write_blog")
graph.add_edge("write_blog", "rate_blog")
graph.add_edge("rate_blog", END)

# compile the graph
workflow=graph.compile()


In [32]:
initial_state={"topic": "Rise of AI in pakistan"}
final_state=workflow.invoke(initial_state)
# print(final_state)

In [33]:
print(final_state["rating"])

8
